In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta8"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta8"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha5"

Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json

Installed package Plotly.NET version 2.0.0-beta8

Installed package FSharp.Stats version 0.4.1

Installed package Plotly.NET.Interactive version 2.0.0-beta8

Installed package Deedle.DotNet.Interactive.Extension version 0.1.0-alpha5

Installed package Deedle version 2.3.0

Loading extensions from `Deedle.DotNet.Interactive.Extension.dll`

Added DeedleFormatterExtension including formatters for Frame and Series

Loading extensions from `Plotly.NET.Interactive.dll`

Added Kernel Extension including formatters for GenericChart

In [ ]:
open Deedle
open Plotly.NET
open System
open System.Net.Http
open System.Net.Http.Json
open System.Text.Json
open System.Text.Json.Serialization

In [ ]:
let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

,Ticks,Value
,System.Int32,System.Int32
01.01.2015 01:00:00,1,5946
01.01.2015 02:00:00,2,5726
01.01.2015 03:00:00,3,5347
01.01.2015 04:00:00,4,5249
01.01.2015 05:00:00,5,5309
01.01.2015 06:00:00,6,5574
01.01.2015 07:00:00,7,5925
01.01.2015 08:00:00,8,6343
01.01.2015 09:00:00,9,6882


In [ ]:
let baseUrl = "http://localhost:5000"

let client = new HttpClient()
client.BaseAddress <- Uri(baseUrl)

client.GetAsync("/")
|> Async.AwaitTask
|> Async.RunSynchronously

Name,Value
traceparent,00-032c7deaac75834290de27ca31301e74-23b786bde788124d-00
Name,Value
Date,"Sun, 02 May 2021 20:23:40 GMT"
Server,Kestrel
Content-Type,text/plain; charset=utf-8
Content-Length,18


In [ ]:
type LinearForecastResponse =
    { LoadForecast: float }

In [ ]:
let from = DateTime.Parse("2019-03-10T00:00:00")
let horizon = 24 * 10

let response =
    client.PostAsJsonAsync("/linear-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues =
    response.Content.ReadFromJsonAsync<LinearForecastResponse array>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> Array.map (fun lf -> lf.LoadForecast)

forecastedValues
|> Array.take 10

index,value
0,6119.9927
1,6120.0117
2,6120.0303
3,6120.0493
4,6120.068
5,6120.087
6,6120.1055
7,6120.1245
8,7563.0146
9,7563.0337


In [ ]:
let keys, actualValues =
    data?Value
    |> Series.observations
    |> Seq.skipWhile (fun (k, _) -> k <> from)
    |> Seq.take horizon
    |> Seq.unzip

let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

In [ ]:
let response =
    client.PostAsJsonAsync("/ssa-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues =
    response.Content.ReadFromJsonAsync<AlternativeLoadForecast>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> fun forecast -> forecast.Forecast

forecastedValues
|> Array.take 10

index,value
0,5962.7
1,5842.6177
2,5793.2554
3,5848.8706
4,6055.1455
5,6183.0723
6,6658.0464
7,7129.324
8,7675.6523
9,7886.1777


In [ ]:
let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

<!-- Plotly chart will be drawn inside this DIV -->